# Setup

In [109]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras import layers, models, optimizers, callbacks

In [110]:
print("TensorFlow:", tf.__version__)
print("Device:", 'GPU' if tf.config.list_physical_devices('GPU') else 'CPU')

TensorFlow: 2.11.1
Device: CPU


# -------------------- CONFIG --------------------

In [111]:
IMG_SIZE        = 64
BATCH_AE        = 512
BATCH_CLF       = 512

# AE pretrain
EPOCHS_AE       = 10 #tested with 5, 10, 20

# Classifier training (A: freeze then unfreeze)
FREEZE_EPOCHS   = 10
UNFREEZE_EPOCHS = 10
LR_FREEZE       = 1e-3
LR_UNFREEZE     = 5e-4

# Classifier training (B: no freezing)
EPOCHS_CLF_NOFREEZE = 10 #tested with 5, 10, 20
LR_NOFREEZE         = 1e-3


# Classifier training (C: frozen encoder)
FREEZE_EPOCHS_C   = 50


VAL_SPLIT       = 0.15
SEED            = 42 #tested with 1, 42, 100 for reproducibility

# Balancing targets (training only)
MIN_DEFECT_TRAIN = 3000    # minimum per defect class in training
MAX_NONE_TRAIN   = 20000   # maximum 'None' used for training


# Data

In [112]:

ROOT_PROC = Path('processed_images')
ROOT_DIST = Path('processed_aux/dist')
ROOT_BAD  = Path('processed_masks/bad')
META_CSV  = Path('converted_images/metadata.csv')

CLASSES  = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-Full','None']
CLS2ID   = {c:i for i,c in enumerate(CLASSES)}
N_CLASSES = len(CLASSES)
DEFECT_CLASSES = [c for c in CLASSES if c != 'None']

OUT_DIR = Path('ae_all_experiments')
OUT_DIR.mkdir(parents=True, exist_ok=True)
ENCODER_PATH = OUT_DIR / 'encoder.keras'

AUTOTUNE = tf.data.AUTOTUNE
rng = np.random.default_rng(SEED)

# -------------------- Dataset (labeled only; 3-channel preprocessing) --------------------

In [113]:
def canon_label(s):
    if s is None: return 'Unlabeled'
    s = str(s).strip().strip('_').replace('_','-').lower()
    table = {
        'center':'Center','donut':'Donut','edge-loc':'Edge-Loc','edge-ring':'Edge-Ring',
        'loc':'Loc','random':'Random','scratch':'Scratch','near-full':'Near-Full',
        'none':'None','unlabeled':'Unlabeled'
    }
    return table.get(s, 'Unlabeled')

assert META_CSV.exists(), "metadata.csv not found at converted_images/metadata.csv"
meta = pd.read_csv(META_CSV)
for col in ['file','label','split']:
    if col not in meta.columns:
        raise ValueError(f"metadata.csv missing required column: {col}")
meta['label'] = meta['label'].apply(canon_label)

def triplet_paths(rel):
    p = Path(rel)
    return (ROOT_PROC/p, ROOT_DIST/p, ROOT_BAD/p)

rows = []
for _, r in meta.iterrows():
    w, d, b = triplet_paths(r['file'])
    rows.append([str(w), str(d), str(b), r['label'], r['split']])
df = pd.DataFrame(rows, columns=['wafer','dist','bad','label','split'])

mask = df['wafer'].apply(lambda p: Path(p).exists()) \
     & df['dist'].apply(lambda p: Path(p).exists()) \
     & df['bad'].apply(lambda p: Path(p).exists())
df = df[mask].reset_index(drop=True)

df_l = df[df['label'].isin(CLASSES)].reset_index(drop=True)  # labeled only


# Train/Test split (uses metadata if present)

In [114]:
train_df = df_l[df_l['split'].astype(str).str.lower().str.startswith('train')].reset_index(drop=True)
test_df  = df_l[df_l['split'].astype(str).str.lower().str.startswith('test')].reset_index(drop=True)
if len(train_df) == 0 or len(test_df) == 0:
    print("INFO: No usable train/test in metadata. Creating a stratified 80/20 split.")
    train_df, test_df = train_test_split(df_l, test_size=0.2, random_state=SEED, stratify=df_l['label'])
    train_df = train_df.reset_index(drop=True)
    test_df  = test_df.reset_index(drop=True)

print(f"Train labeled: {len(train_df)} | Test labeled: {len(test_df)}")

INFO: No usable train/test in metadata. Creating a stratified 80/20 split.
Train labeled: 138360 | Test labeled: 34590


# -------------------- Balance TRAIN (upsample defects, downsample None) --------------------

In [115]:
def balance_supervised_train(train_df,
                             min_defect=MIN_DEFECT_TRAIN,
                             max_none=MAX_NONE_TRAIN):
    none_all = train_df[train_df['label'] == 'None'].reset_index(drop=True)
    defects_all = train_df[train_df['label'] != 'None'].reset_index(drop=True)

    # Downsample None to at most max_none
    if len(none_all) > max_none:
        none_bal = none_all.sample(max_none, random_state=SEED).reset_index(drop=True)
    else:
        none_bal = none_all

    # Upsample each defect to at least min_defect
    blocks = []
    for c in DEFECT_CLASSES:
        sub = defects_all[defects_all['label'] == c].reset_index(drop=True)
        if len(sub) == 0:
            continue
        if len(sub) < min_defect:
            need = min_defect - len(sub)
            supl = sub.sample(need, replace=True, random_state=SEED)
            sub_bal = pd.concat([sub, supl], axis=0).reset_index(drop=True)
        else:
            sub_bal = sub
        blocks.append(sub_bal)

    defects_bal = pd.concat(blocks, axis=0).reset_index(drop=True)
    train_bal = pd.concat([defects_bal, none_bal], axis=0).sample(frac=1.0, random_state=SEED).reset_index(drop=True)
    return train_bal

train_df_bal = balance_supervised_train(train_df)
print("\nBalanced TRAIN counts:")
print(train_df_bal['label'].value_counts().reindex(CLASSES).fillna(0).astype(int))


Balanced TRAIN counts:
label
Center        3435
Donut         3000
Edge-Loc      4151
Edge-Ring     7744
Loc           3000
Random        3000
Scratch       3000
Near-Full     3000
None         20000
Name: count, dtype: int32


# -------------------- Preprocess & tf.data --------------------

In [116]:
def read_gray64(path):
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(path)
    if img.shape[0] != IMG_SIZE or img.shape[1] != IMG_SIZE:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
    return img

def load_triplet_row(row):
    w = read_gray64(row['wafer'])
    d = read_gray64(row['dist'])
    b = read_gray64(row['bad'])
    # wafer: {0,127,255} -> {0,0.5,1}
    w = (w // 127).astype(np.float32) / 2.0
    d = (d.astype(np.float32) / 255.0)
    b = (b.astype(np.float32) / 255.0)
    x = np.stack([w, d, b], axis=-1).astype(np.float32)  # H,W,3
    return x

def gen_x(frame):
    for _, r in frame.iterrows():
        yield load_triplet_row(r)

def gen_xy(frame):
    for _, r in frame.iterrows():
        yield load_triplet_row(r), CLS2ID[r['label']]

def make_ds_ae(frame, batch, shuffle=True):
    ds_x = tf.data.Dataset.from_generator(
        lambda: gen_x(frame),
        output_signature=tf.TensorSpec(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
    )
    if shuffle:
        ds_x = ds_x.shuffle(4096, seed=SEED)
    ds = ds_x.map(lambda x: (x, x), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

def make_ds_xy(frame, batch, shuffle=True):
    ds = tf.data.Dataset.from_generator(
        lambda: gen_xy(frame),
        output_signature=(
            tf.TensorSpec(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32),
        )
    )
    if shuffle:
        ds = ds.shuffle(4096, seed=SEED)
    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

In [117]:
train_bal, val_bal = train_test_split(
    train_df_bal, test_size=VAL_SPLIT, random_state=SEED, stratify=train_df_bal['label']
)
train_bal = train_bal.reset_index(drop=True)
val_bal   = val_bal.reset_index(drop=True)
print(f"\nBalanced splits ( Train: {len(train_bal)} | Val: {len(val_bal)})")


Balanced splits ( Train: 42780 | Val: 7550)


# Build datasets

In [118]:
ae_train_ds = make_ds_ae(train_bal, BATCH_AE, shuffle=True)
ae_val_ds   = make_ds_ae(val_bal,   BATCH_AE, shuffle=False)

In [119]:
clf_train_ds = make_ds_xy(train_bal, BATCH_CLF, shuffle=True)
clf_val_ds   = make_ds_xy(val_bal,   BATCH_CLF, shuffle=False)
test_ds      = make_ds_xy(test_df,   BATCH_CLF, shuffle=False)

# -------------------- Autoencoder --------------------

# Models

In [120]:
def conv_block(x, filters, name_prefix):
    x = layers.Conv2D(filters, 3, padding='same', use_bias=False, name=f'{name_prefix}_conv')(x)
    x = layers.BatchNormalization(name=f'{name_prefix}_bn')(x)
    x = layers.ReLU(name=f'{name_prefix}_relu')(x)
    return x

def build_autoencoder_big():
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input')
    # Encoder 
    x = conv_block(inp, 64,  'enc1_1')
    x = conv_block(x,  64,  'enc1_2')
    x = layers.MaxPooling2D(name='enc1_pool')(x)           # 32x32

    x = conv_block(x,  128, 'enc2_1')
    x = conv_block(x,  128, 'enc2_2')
    x = layers.MaxPooling2D(name='enc2_pool')(x)           # 16x16

    x = conv_block(x,  128, 'enc3_1')
    x = conv_block(x,  128, 'enc3_2')
    latent = layers.MaxPooling2D(name='latent')(x)         # 8x8 (bottleneck)

    # Decoder
    y = conv_block(latent, 128, 'dec1_pre')
    y = layers.Conv2DTranspose(128, 3, strides=2, padding='same', use_bias=False, name='dec1_deconv')(y)  # 16x16
    y = layers.BatchNormalization(name='dec1_bn')(y)
    y = layers.ReLU(name='dec1_relu')(y)

    y = conv_block(y, 64, 'dec2_pre')
    y = layers.Conv2DTranspose(64, 3, strides=2, padding='same', use_bias=False, name='dec2_deconv')(y)   # 32x32
    y = layers.BatchNormalization(name='dec2_bn')(y)
    y = layers.ReLU(name='dec2_relu')(y)

    y = layers.Conv2DTranspose(32, 3, strides=2, padding='same', use_bias=False, name='dec3_deconv')(y)   # 64x64
    y = layers.BatchNormalization(name='dec3_bn')(y)
    y = layers.ReLU(name='dec3_relu')(y)

    recon = layers.Conv2D(3, 3, padding='same', activation='sigmoid', name='recon')(y)
    ae = models.Model(inp, recon, name='autoencoder_big')
    ae.compile(optimizer=optimizers.Adam(1e-3), loss='mse')
    return ae

# === AE training & encoder saving ===

In [121]:
ae = build_autoencoder_big()
ae.summary()

Model: "autoencoder_big"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 enc1_1_conv (Conv2D)        (None, 64, 64, 64)        1728      
                                                                 
 enc1_1_bn (BatchNormalizati  (None, 64, 64, 64)       256       
 on)                                                             
                                                                 
 enc1_1_relu (ReLU)          (None, 64, 64, 64)        0         
                                                                 
 enc1_2_conv (Conv2D)        (None, 64, 64, 64)        36864     
                                                                 
 enc1_2_bn (BatchNormalizati  (None, 64, 64, 64)       256       
 on)                                               

In [122]:
ae.fit(
    ae_train_ds,
    validation_data=ae_val_ds,
    epochs=EPOCHS_AE,
    verbose=1,
    callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)
encoder = models.Model(ae.input, ae.get_layer('latent').output, name='encoder')
encoder.save(ENCODER_PATH)
print("Saved encoder:", ENCODER_PATH)

Epoch 1/10
84/84 [==============================] - 2027s 23s/step - loss: 0.0241 - val_loss: 0.0573
Epoch 2/10
84/84 [==============================] - 1648s 19s/step - loss: 0.0090 - val_loss: 0.0502
Epoch 3/10
84/84 [==============================] - 1253s 14s/step - loss: 0.0075 - val_loss: 0.0379
Epoch 4/10
84/84 [==============================] - 2012s 23s/step - loss: 0.0069 - val_loss: 0.0257
Epoch 5/10
84/84 [==============================] - 1872s 22s/step - loss: 0.0063 - val_loss: 0.0167
Epoch 6/10
84/84 [==============================] - 1713s 20s/step - loss: 0.0059 - val_loss: 0.0084
Epoch 7/10
84/84 [==============================] - 1776s 21s/step - loss: 0.0055 - val_loss: 0.0063
Epoch 8/10
84/84 [==============================] - 1459s 17s/step - loss: 0.0051 - val_loss: 0.0059
Epoch 9/10
84/84 [==============================] - 1800s 21s/step - loss: 0.0047 - val_loss: 0.0053
Epoch 10/10
84/84 [==============================] - 1906s 22s/step - loss: 0.0044 - val_lo

# -------------------- Classifier head builder --------------------

In [123]:
def build_classifier_from_encoder(encoder_model, dense_units=256, dropout=0.25):
    clf_in = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input')
    feat   = encoder_model(clf_in)
    gap    = layers.GlobalAveragePooling2D(name='gap')(feat)
    h      = layers.Dense(dense_units, activation='relu', name='head_dense')(gap)
    h      = layers.Dropout(dropout, name='head_drop')(h)
    logits = layers.Dense(N_CLASSES, activation=None, name='logits')(h)
    prob   = layers.Softmax(name='softmax')(logits)
    clf    = models.Model(clf_in, prob, name='classifier_big')
    return clf

# -------------------- Utility: Evaluate on ORIGINAL test split --------------------

In [124]:
def eval_and_print(model, test_ds, tag):
    y_true, y_pred = [], []
    for xb, yb in test_ds:
        p = model.predict(xb, verbose=0)
        y_true += yb.numpy().tolist()
        y_pred += p.argmax(axis=1).tolist()
    overall_acc = accuracy_score(y_true, y_pred)
    macro_f1    = f1_score(y_true, y_pred, average='macro')
    print(f"\n=== Test Results ({tag}) ===")
    print("Overall accuracy:", f"{overall_acc:.4f}")
    print("Macro-F1        :", f"{macro_f1:.4f}")
    print(classification_report(y_true, y_pred, target_names=CLASSES, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

# -------------------- EXPERIMENT A — Freeze then Unfreeze --------------------

In [ ]:
print("\n================ EXPERIMENT A: Freeze_Unfreeze ================")
enc_A = tf.keras.models.load_model(ENCODER_PATH, compile=False)
for layer in enc_A.layers:
    layer.trainable = False
clf_A = build_classifier_from_encoder(enc_A, dense_units=384, dropout=0.3)
clf_A.compile(optimizer=optimizers.Adam(LR_FREEZE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
clf_A.fit(clf_train_ds, validation_data=clf_val_ds, epochs=FREEZE_EPOCHS, verbose=1)
for layer in enc_A.layers:
    layer.trainable = True
clf_A.compile(optimizer=optimizers.Adam(LR_UNFREEZE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
clf_A.fit(clf_train_ds, validation_data=clf_val_ds, epochs=UNFREEZE_EPOCHS, verbose=1,
          callbacks=[callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)])
eval_and_print(clf_A, test_ds, tag="A: Freeze_Unfreeze")


================ EXPERIMENT A: Freeze_Unfreeze ================
Epoch 1/10
84/84 [==============================] - 232s 2s/step - loss: 1.5999 - accuracy: 0.4679 - val_loss: 1.4167 - val_accuracy: 0.5230
Epoch 2/10
84/84 [==============================] - 216s 3s/step - loss: 1.3538 - accuracy: 0.5395 - val_loss: 1.2746 - val_accuracy: 0.5502
Epoch 3/10
84/84 [==============================] - 21909s 264s/step - loss: 1.2549 - accuracy: 0.5741 - val_loss: 1.2073 - val_accuracy: 0.5887
Epoch 4/10
84/84 [==============================] - 165s 2s/step - loss: 1.1955 - accuracy: 0.6045 - val_loss: 1.1481 - val_accuracy: 0.6184
Epoch 5/10
83/84 [============================>.] - ETA: 1s - loss: 1.1505 - accuracy: 0.6282

# -------------------- EXPERIMENT B — No Freezing (fresh encoder) --------------------

In [ ]:
print("\n================ EXPERIMENT B: No Freezing ================")
enc_B = tf.keras.models.load_model(ENCODER_PATH, compile=False)  # reuse saved encoder
for layer in enc_B.layers:
    layer.trainable = True
clf_B = build_classifier_from_encoder(enc_B, dense_units=384, dropout=0.3)
clf_B.compile(optimizer=optimizers.Adam(LR_NOFREEZE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
clf_B.fit(clf_train_ds, validation_data=clf_val_ds, epochs=EPOCHS_CLF_NOFREEZE, verbose=1,
          callbacks=[callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

eval_and_print(clf_B, test_ds, tag="B: No Freezing")

In [ ]:
def build_classifier3_from_encoder(encoder_model, dense_units=256, dropout=0.25):
    clf_in = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input')
    feat   = encoder_model(clf_in)
    gap    = layers.GlobalAveragePooling2D(name='gap')(feat)
    
    h      = layers.Dense(dense_units, activation='relu', name='head_dense_1')(gap)
    
    h      = layers.Dense(dense_units // 2, activation='relu', name='head_dense_2')(h)
    
    h      = layers.Dropout(dropout, name='head_drop')(h)
    
    logits = layers.Dense(N_CLASSES, activation=None, name='logits')(h)
    prob   = layers.Softmax(name='softmax')(logits)
    
    clf    = models.Model(clf_in, prob, name='classifier_bigger')
    return clf

# -------------------- EXPERIMENT C: Freezen Encoder --------------------

In [ ]:
print("\n================ EXPERIMENT C: Freezen Encoder ================")
enc_A = tf.keras.models.load_model(ENCODER_PATH, compile=False)
for layer in enc_A.layers:
    layer.trainable = False
clf_A = build_classifier3_from_encoder(enc_A, dense_units=384, dropout=0.3)
clf_A.compile(optimizer=optimizers.Adam(LR_FREEZE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
clf_A.fit(clf_train_ds, validation_data=clf_val_ds, epochs=FREEZE_EPOCHS_C, verbose=1)


================ EXPERIMENT A: Freeze → Unfreeze ================
Epoch 1/50
84/84 [==============================] - 1063s 12s/step - loss: 1.4788 - accuracy: 0.5103 - val_loss: 1.2637 - val_accuracy: 0.5694
Epoch 2/50
84/84 [==============================] - 138s 1s/step - loss: 1.2432 - accuracy: 0.5842 - val_loss: 1.1565 - val_accuracy: 0.6127
Epoch 3/50
84/84 [==============================] - 135s 2s/step - loss: 1.1431 - accuracy: 0.6270 - val_loss: 1.0619 - val_accuracy: 0.6511
Epoch 4/50
84/84 [==============================] - 144s 2s/step - loss: 1.0698 - accuracy: 0.6558 - val_loss: 0.9912 - val_accuracy: 0.6804
Epoch 5/50
84/84 [==============================] - 145s 2s/step - loss: 0.9920 - accuracy: 0.6852 - val_loss: 0.9183 - val_accuracy: 0.7205
Epoch 6/50
84/84 [==============================] - 142s 2s/step - loss: 0.9553 - accuracy: 0.6964 - val_loss: 0.9157 - val_accuracy: 0.7233
Epoch 7/50
84/84 [==============================] - 144s 2s/step - loss: 0.9056 - acc

In [ ]:
def build_classifier3_no_encoder(dense_units=256, dropout=0.25):
    clf_in = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input')
    gap    = tf.keras.layers.GlobalAveragePooling2D(name='gap')(clf_in)
    h      = tf.keras.layers.Dense(dense_units, activation='relu', name='head_dense_1')(gap)
    h      = tf.keras.layers.Dense(dense_units // 2, activation='relu', name='head_dense_2')(h)
    h      = tf.keras.layers.Dropout(dropout, name='head_drop')(h)
    logits = tf.keras.layers.Dense(N_CLASSES, activation=None, name='logits')(h)
    prob   = tf.keras.layers.Softmax(name='softmax')(logits)
    return tf.keras.Model(clf_in, prob, name='classifier_head_only')


In [ ]:
print("\n================ EXPERIMENT A: No-Encoder =================")

# Build the classifier head that takes raw images
clf_Ab = build_classifier3_no_encoder(dense_units=384, dropout=0.3)

# Compile (same optimizer/loss/metrics)
clf_Ab.compile(
    optimizer=optimizers.Adam(LR_FREEZE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
clf_Ab.fit(
    clf_train_ds,
    validation_data=clf_val_ds,
    epochs=FREEZE_EPOCHS,
    verbose=1
)



================ EXPERIMENT A: No-Encoder =================
Epoch 1/50
84/84 [==============================] - 634s 7s/step - loss: 1.7580 - accuracy: 0.4370 - val_loss: 1.6138 - val_accuracy: 0.4583
Epoch 2/50
84/84 [==============================] - 98s 924ms/step - loss: 1.5587 - accuracy: 0.4740 - val_loss: 1.4668 - val_accuracy: 0.4968
Epoch 3/50
84/84 [==============================] - 43s 478ms/step - loss: 1.4417 - accuracy: 0.5063 - val_loss: 1.3831 - val_accuracy: 0.5212
Epoch 4/50
84/84 [==============================] - 43s 473ms/step - loss: 1.3793 - accuracy: 0.5283 - val_loss: 1.3334 - val_accuracy: 0.5372
Epoch 5/50
84/84 [==============================] - 42s 468ms/step - loss: 1.3394 - accuracy: 0.5418 - val_loss: 1.2999 - val_accuracy: 0.5483
Epoch 6/50
84/84 [==============================] - 46s 500ms/step - loss: 1.3103 - accuracy: 0.5568 - val_loss: 1.2763 - val_accuracy: 0.5603
Epoch 7/50
84/84 [==============================] - 42s 465ms/step - loss: 1.2894 -